### Imports

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import get_scheduler

### Load Dataset

In [15]:
dataset = pd.read_pickle('Data/small_test_set.pkl')
dataset

,title,date,ticker,returns
219534,"Benzinga’s Top Upgrades (SBNY, CCL, MPW, MA, F...",2010-05-21 08:27:00,CCL,0.013952
912659,"Top Performing Industries For August 9, 2016",2016-08-09 10:54:00,NTL,-0.010309
165581,Bridgeline DIgital Reports Q3 Loss $0.26 Vs Es...,2015-08-14 08:01:00,BLIN,-0.048276
1355251,"Earnings Scheduled For March 24, 2015",2015-03-24 04:04:00,WSCI,0.001757
362331,Puts Purchased on Dick's Sporting Goods (DKS),2011-01-06 12:40:00,DKS,-0.045442
...,...,...,...,...
1236358,Standpoint Research Downgrades Tempur-pedic In...,2013-10-01 11:17:00,TPX,0.023658
1370405,"Sector Update: Utilities Leading, Consumer Goo...",2011-08-24 10:36:00,XLF,0.026678
90206,Aramark Acquires On-Demand Food Delivery Servi...,2019-08-06 06:44:00,ARMK,0.047018
1342675,Wheeler Real Estate Investment Trust Responds ...,2018-03-16 04:19:00,WHLR,0.064220


### Split Dataset into Train, Validation, and Test
- Train (80%)
- Test (10%)
- Validation (10%)

In [12]:
def split(df):
    x_train, x_test_and_val, y_train, y_test_and_val  = train_test_split(df['title'], df['returns'], random_state=42, test_size=0.2) # train set is 80%,
    x_test, x_val, y_test, y_val = train_test_split(x_test_and_val, y_test_and_val, test_size=0.5, random_state=42) # test and val are 50% of the remaining 20% = 10%. 
    return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = split(dataset)

### Finbert Model Training

In [14]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
#Predict function from the previuos homework modified to work with the Question and Format
def predict_samples(titles, model, tokenizer):
    predictions = []
 
    for titles in tqdm(zip(contexts, questions), total=len(titles)):
        # Tokenize the samples
        input = tokenizer(titles, padding=True, truncation=True, return_tensors="pt")
        input.to(model.device)
        # Generate output
        output = model(**input)
        
        prediction = tokenizer.decode(output, skip_special_tokens=True)
        predictions.append(prediction)

    return predictions

In [ ]:


optimizer = AdamW(model.parameters(), lr=5e-5)



num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

CAVIN WORK

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

df = dataset

# Tokenize and preprocess the input data
tokenized_inputs = tokenizer(df['article_titles'].tolist(), padding=True, truncation=True, return_tensors='tf')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tokenized_inputs, df['stock_returns'], test_size=0.2, random_state=42)

# Build a model architecture
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
outputs = bert_model(input_ids)[1]  # Extract pooled output
outputs = tf.keras.layers.Dense(1, activation='linear')(outputs)  # Regression layer

model = tf.keras.Model(inputs=input_ids, outputs=outputs)
model.compile(optimizer='adam', loss='mse')  # Mean Squared Error loss for regression

# Train the model
model.fit(X_train['input_ids'], y_train, epochs=5, validation_data=(X_test['input_ids'], y_test))

# Evaluate the model
mse = model.evaluate(X_test['input_ids'], y_test)
print(f'Mean Squared Error on test set: {mse}')

# Make predictions
predictions = model.predict(X_test['input_ids'])


BERT Model Training